## SI Calculation: an example using generated sample trades

Here we will generate a few sample trades from the RTS 2 Annex III taxonomy, enrich the trades with EUR notionals and run a very simple SI calculation.

The SI calculation includes a number of tests.  See the official word:
https://ec.europa.eu/transparency/regdoc/rep/3/2016/EN/3-2016-2398-EN-F1-1.PDF


In [1]:
import rts2_annex3
import random
import json

root = rts2_annex3.class_root

asset_class = root.asset_class_by_name("Credit Derivatives")

# Ask the Asset class to generate some sample trade
sample_trades = asset_class.make_test_samples(number=50)

# Print the one of the generated trades
print(vars(random.choice(sample_trades)))


{'sub_asset_class_name': 'Index credit default swap (CDS)', 'notional_currency': 'notional_currency.value', 'to_date': datetime.date(2019, 4, 23), 'asset_class_name': 'Credit Derivatives', 'underlying_index': 'underlying_index.value', 'from_date': datetime.date(2018, 4, 23)}


## LEIs

In a real firm with real trades we would need to know the LEI (Legal Entity Identifier) of the legal entity which did each trade because SI status is reported distinctly for each legal entity (LEI).

Quite often firms will do trades within a single legal entity, perhaps to move risk from one trading desk to another.  These are called intra-entity trades and must be filtered out before the SI calculation.  For this example we'll say that all the trades we generated are inter-entity trades, trades between distinct legal entities, so we count them all.

In this example we'll use just one LEI, and not even a valid one, but it will suffice for the example.

In [2]:
# Add our LEI to each trade
our_lei = 'Our_LEI_here'
for sample_trade in sample_trades:
    sample_trade.our_lei = our_lei

# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'sub_asset_class_name': 'Bespoke basket credit default swap (CDS)', 'asset_class_name': 'Credit Derivatives', 'our_lei': 'Our_LEI_here'}


## Trade Date
The SI calculation includes checks for frequency, the number of trades done in a single week.  To work that out we need a trade date for each trade.  Here we'll just use a few dates and add these to our sample trades.

In [3]:
# We give each sample trade a trade date in a 30 day range of dates
import datetime

sample_dates = []
today = datetime.date.today()
for day_number in range(-30, 0):
    a_date =  today + datetime.timedelta(day_number)
    if a_date.weekday() < 6:
        sample_dates.append(a_date)

for sample_trade in sample_trades:
    sample_trade.trade_date = random.choice(sample_dates)
    
# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'sub_asset_class_name': 'Bespoke basket credit default swap (CDS)', 'trade_date': datetime.date(2018, 3, 29), 'asset_class_name': 'Credit Derivatives', 'our_lei': 'Our_LEI_here'}


## MIC
The Market Identifier Code (MIC) is the ISO 10383 ID for a trading venue, for example a stock exchange.  The regulator is expected to provide a list of MIC values which identify venues which are recognised for the purposes of the SI calculation.  Trades which are done on vs. off recognised venues are counted differently.

In [4]:
# We define our MICs.  A MIC value is always 4 charcters in length.  The values used
# here are made-up nonsense, but good enough for an illustration

eea_mics = ['EEA1', 'EEA2', 'EEA3']
non_eea_mics = ['OFF1', 'OFF2', 'OFF3', 'OFF4']
all_mics = eea_mics + non_eea_mics

# Add a MIC to each sample trade
for sample_trade in sample_trades:
    sample_trade.mic = random.choice(all_mics)

# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'sub_asset_class_name': 'Other credit derivatives', 'trade_date': datetime.date(2018, 3, 30), 'mic': 'OFF4', 'asset_class_name': 'Credit Derivatives', 'our_lei': 'Our_LEI_here'}


## Own Account

We need to know if a trade was done on the firms own account in response to a client order.  Such trades are counted differently. 

In [5]:
# Own Account is simply a boolean.  Either this is a trade which the regulator views
# as being on own account in response to a client order, or not.  I use a random
# boolean.  It's more likely the firm will trade with a clients money that their
# own, so ...

own_account_booleans = [True, False, False, False]

for sample_trade in sample_trades:
    sample_trade.own_account = random.choice(own_account_booleans)
    
# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'sub_asset_class_name': 'Index credit default swap (CDS)', 'trade_date': datetime.date(2018, 4, 12), 'mic': 'OFF4', 'asset_class_name': 'Credit Derivatives', 'own_account': False, 'notional_currency': 'notional_currency.value', 'to_date': datetime.date(2021, 1, 17), 'our_lei': 'Our_LEI_here', 'underlying_index': 'underlying_index.value', 'from_date': datetime.date(2018, 4, 23)}


## EUR Notional
Another measure used by the SI calculation is the EUR notional value of each trade.  Here we assign a notional value to each trade.

In [6]:
# Add a random-ish Euro Notional amount of n million EUR to each trade
notional_amounts = [x * 1000000 for x in [1, 1, 1, 2, 2, 5, 10, 25]]

for sample_trade in sample_trades:
    sample_trade.eur_notional = random.choice(notional_amounts)

# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'sub_asset_class_name': 'Single name CDS options', 'trade_date': datetime.date(2018, 4, 13), 'cds_sub_class': 'cds_sub_class.value', 'mic': 'OFF4', 'asset_class_name': 'Credit Derivatives', 'own_account': False, 'to_date': datetime.date(2018, 10, 20), 'our_lei': 'Our_LEI_here', 'eur_notional': 1000000, 'from_date': datetime.date(2018, 4, 23)}


## RTS 2 Annex III Classification
The last step before we start the SI calculation is to add the RTS 2 Annex III classification to each trade.

In [7]:
# Now classify each trade and add the JSON classification back to the trade
for sample_trade in sample_trades:
    classification = root.classification_for(subject=sample_trade)
    json_classification = json.dumps(classification.classification_dict(), indent=4)
    sample_trade.rts2_classification = json_classification

print(random.choice(sample_trades).rts2_classification)



{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Credit Derivatives",
    "Sub-asset class": "CDS index options",
    "Segmentation criterion 1 description": "CDS index sub-class as specified for the sub-asset class of index credit default swap (CDS )",
    "Segmentation criterion 1": "cds_index_sub_class.value",
    "Segmentation criterion 2 description": "time maturity bucket of the option defined as follows:",
    "Segmentation criterion 2": "Maturity bucket 2: 6 months to 1 year"
}


## SI Calculation
The next step is to run the SI Calculation.  This is more a series of grouping exercises than a real calculation, but here we go ...